In [6]:
from pyspark.sql import SparkSession

session = SparkSession.builder \
    .appName("MLFlow-Spark-Integration") \
    .master("local[*]") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .getOrCreate()

session

import sklearn.datasets as datasets
import mlflow
from mlflow.pyfunc import spark_udf as mlflow_udf

# Connecting to an MLFlow server
mlflow.set_tracking_uri("http://localhost:5000")

# Creating a Spark UDF 
apply_model = mlflow_udf(session, "models:/IrisModel/1")

apply_model

<function mlflow.pyfunc.spark_udf.<locals>.predict(*args)>

In [8]:
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split
import pandas as pd

iris = datasets.load_iris()

x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target)

pandas_df = pd.DataFrame(x_test, columns = iris.feature_names)

x = session.createDataFrame(pandas_df)

pandas_df.head(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,6.7,3.0,5.0,1.7
1,5.6,2.5,3.9,1.1
2,6.5,3.2,5.1,2.0
3,7.9,3.8,6.4,2.0
4,4.9,3.0,1.4,0.2
5,5.7,3.0,4.2,1.2
6,5.8,2.7,5.1,1.9
7,5.8,2.8,5.1,2.4
8,6.7,3.3,5.7,2.5
9,5.0,3.3,1.4,0.2


In [9]:
# Using the MLFlow UDF on the DataFrame
x.withColumn(
    "prediction",
    apply_model("sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)")
).show(10)

Py4JJavaError: An error occurred while calling o266.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/klok/Apps/Spark/2.4.4/python/lib/pyspark.zip/pyspark/worker.py", line 267, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 3.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:456)
	at org.apache.spark.sql.execution.python.ArrowPythonRunner$$anon$1.read(ArrowPythonRunner.scala:172)
	at org.apache.spark.sql.execution.python.ArrowPythonRunner$$anon$1.read(ArrowPythonR